In [ ]:
import requests
import mysql.connector
import time
from datetime import datetime, timedelta

# ----------------------------
# Configuration
# ----------------------------
API_KEY = 'Eo2TeVMWK8DSHA5Nra4Mrb4bjg2UjmJub72faW2J'
START_DATE = datetime.strptime("2024-01-01", "%Y-%m-%d")
MAX_RECORDS = 10000

# ----------------------------
# Safe Conversion Functions
# ----------------------------

def safe_int(val):
    try: return int(val) if val is not None else None
    except: return None

def safe_float(val):
    try: return float(val) if val is not None else None
    except: return None

def safe_str(val):
    return str(val) if val is not None else None

def safe_bool(val):
    return bool(val) if val is not None else None

def safe_date(val, fmt='%Y-%m-%d'):
    try: return datetime.strptime(val, fmt).date() if val else None
    except: return None

# ----------------------------
# Connect and Setup MySQL
# ----------------------------

def connect_and_setup_mysql():
    conn = mysql.connector.connect(
        host='',
        user='',  
        password='',  
    )
    cursor = conn.cursor()

    # Create database
    cursor.execute("CREATE DATABASE IF NOT EXISTS NEO_DB")
    cursor.execute("USE NEO_DB")

    # Create 'asteroids' table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS asteroids (
            id INT PRIMARY KEY,
            name VARCHAR(255),
            absolute_magnitude_h FLOAT,
            estimated_diameter_min_km FLOAT,
            estimated_diameter_max_km FLOAT,
            is_potentially_hazardous_asteroid BOOLEAN
        )
    ''')

    # Create 'close_approach' table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS close_approach (
            neo_reference_id INT,
            close_approach_date DATE,
            relative_velocity_kmph FLOAT,
            astronomical FLOAT,
            miss_distance_km FLOAT,
            miss_distance_lunar FLOAT,
            orbiting_body VARCHAR(255),
            PRIMARY KEY (neo_reference_id, close_approach_date),
                   FOREIGN KEY (neo_reference_id) REFERENCES asteroids(id)
                   ON DELETE CASCADE
                   ON UPDATE CASCADE
        )
    ''')

    conn.commit()
    return conn, cursor

# ----------------------------
# Insert Data into MySQL
# ----------------------------

def insert_data(cursor, conn, asteroid_data, approach_data):
    # Asteroids may have duplicates — insert IGNORE or deduplicate in code
    asteroid_query = '''
        INSERT IGNORE INTO asteroids (
            id, name, absolute_magnitude_h,
            estimated_diameter_min_km, estimated_diameter_max_km,
            is_potentially_hazardous_asteroid
        ) VALUES (%s, %s, %s, %s, %s, %s)
    '''

    approach_query = '''
        INSERT IGNORE INTO close_approach (
            neo_reference_id, close_approach_date,
            relative_velocity_kmph, astronomical,
            miss_distance_km, miss_distance_lunar, orbiting_body
        ) VALUES (%s, %s, %s, %s, %s, %s, %s)
    '''

    cursor.executemany(asteroid_query, asteroid_data)
    cursor.executemany(approach_query, approach_data)
    conn.commit()

# ----------------------------
# Fetch and Clean Data from API
# ----------------------------

def fetch_asteroid_data():
    asteroid_set = set()
    asteroid_data = []
    approach_data = []

    current_date = START_DATE

    while len(approach_data) < MAX_RECORDS:
        end_date = current_date + timedelta(days=6)
        url = (
            f"https://api.nasa.gov/neo/rest/v1/feed?"
            f"start_date={current_date.strftime('%Y-%m-%d')}&"
            f"end_date={end_date.strftime('%Y-%m-%d')}&api_key={API_KEY}"
        )

        print(f"📅 Fetching data from {current_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}... ", end='')


        response = requests.get(url)
        if response.status_code != 200:
            print("Error fetching data:", response.status_code)
            break

        data = response.json()
        neo_objects = data.get('near_earth_objects', {})

        for date in neo_objects:
            for ast in neo_objects[date]:
                if not ast.get('close_approach_data'):
                    continue

                approach = ast['close_approach_data'][0]
                dia = ast.get('estimated_diameter', {}).get('kilometers', {})

                ast_id = safe_int(ast.get('id'))

                # Store asteroid info if not already added
                if ast_id not in asteroid_set:
                    asteroid_set.add(ast_id)
                    asteroid_record = (
                        ast_id,
                        safe_str(ast.get('name')),
                        safe_float(ast.get('absolute_magnitude_h')),
                        safe_float(dia.get('estimated_diameter_min')),
                        safe_float(dia.get('estimated_diameter_max')),
                        safe_bool(ast.get('is_potentially_hazardous_asteroid'))
                    )
                    asteroid_data.append(asteroid_record)

                # Store approach info
                approach_record = (
                    ast_id,
                    safe_date(approach.get('close_approach_date')),
                    safe_float(approach.get('relative_velocity', {}).get('kilometers_per_hour')),
                    safe_float(approach.get('miss_distance', {}).get('astronomical')),
                    safe_float(approach.get('miss_distance', {}).get('kilometers')),
                    safe_float(approach.get('miss_distance', {}).get('lunar')),
                    safe_str(approach.get('orbiting_body'))
                )
                approach_data.append(approach_record)

                if len(approach_data) >= MAX_RECORDS:
                    break
            if len(approach_data) >= MAX_RECORDS:
                break

        print(f"✅ Fetched so far: {len(approach_data)}")
        current_date += timedelta(days=7)
        time.sleep(1)

    return asteroid_data, approach_data

# ----------------------------
# Main Execution
# ----------------------------

def main():
    print("🔄 Starting asteroid data load...")

    # Step 1: Fetch and separate data
    asteroid_data, approach_data = fetch_asteroid_data()
    print(f"✅ Total Asteroids: {len(asteroid_data)}")
    print(f"✅ Total Close Approaches: {len(approach_data)}")

    # Step 2: Setup MySQL
    conn, cursor = connect_and_setup_mysql()

    # Step 3: Insert data
    insert_data(cursor, conn, asteroid_data, approach_data)

    # Step 4: Close connection
    cursor.close()
    conn.close()
    print("✅ Data inserted into 'asteroids' and 'close_approach' tables successfully.")

if __name__ == '__main__':
    main()

# This script fetches asteroid data from NASA's API, processes it, and stores it in a MySQL database.
# It handles potential duplicates and ensures data integrity with safe conversion functions.

# Total Asteroids: 8375
# duplicated skipped -  8375 datas inserted into 'asteroids' table
# duplicated skipped -  10000 datas inserted into 'close_approach' table

# # Note: The 'close_approach' table has a composite primary key on (neo_reference_id, close_approach_date) to ensure that
# each close approach is unique per asteroid and date.
#
# duplicates are avoided/skipped - bcz same asteroid 'id' is came near earth many  times and stored in data with same 'id'
# Total Close Approaches: 10000